In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 

from textblob import TextBlob

In [3]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
import numpy as np
import re

##LDA stuff
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

##cleaning stuff
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim
from gensim.models import CoherenceModel

##plotting
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
# TODO: modify these list if needed (eg. if you want to load only 1 csv from star3, delete other csvs in star3 list)
star3 = ['cleaned_ibis-sg-bencoolen.csv','cleaned_hotel-boss.csv','cleaned_hotel-G.csv',
           'cleaned_village-hotel-albert-court-by-far-east-hospitality.csv',
           'cleaned_holiday-inn-express-clarke-quay.csv']
star4 = ['cleaned_village-hotel-changi-by-far-east-hospitality.csv',
         'cleaned_park-regis.csv', 'cleaned_grand-mercure-sg-roxy.csv',
         'cleaned_paradox-sg-merchant-court.csv','cleaned_crowne-plaza.csv']
star5 = ['cleaned_fullerton.csv', 'cleaned_parkroyal-collection-marina-bay.csv', 'cleaned_pan-pacific.csv',
          'cleaned_mbs_total.csv', 'cleaned_swissotel-the-stamford.csv']

RAW_FOLDER = "../../data/processed/"

def combine_csv_to_dataframe(file_names, all_star = False, filterDate = True):
    """
    Combine multiple CSV files into a single DataFrame.

    Parameters:
    file_names (list): List of CSV file names. 
    all_star (bool): whether or not to load all the hotels (False if only want to load 1 type of hotel star). 
    filterData (bool): whether or not to remove all data dated before 2015

    Returns:
    pd.DataFrame: Combined DataFrame.
    """
    combined_df = pd.DataFrame()

    for file_name in file_names:
        file_interim_path = RAW_FOLDER + file_name
        file_path = file_interim_path
        try:
            df = pd.read_csv(file_path)
            if all_star:
                if file_name in star3:
                    df["star"] = 3
                elif file_name in star4:
                    df["star"] = 4
                else:
                    df["star"] = 5
            #print(f"Length of {file_name} is {len(df)}")
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            #print(len(combined_df))
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"Empty or invalid CSV file: {file_name}")
            
    combined_df = combined_df[combined_df['date'] >= '2015-01-01']
                                
    return combined_df
data = combine_csv_to_dataframe(star3+star4+star5, all_star = True, filterDate = True)
#data[['traveller_username','date','travel_type','traveller_total_contributions','traveller_total_helpful_contributions','review_title','review_text','rating']].head(5)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 68292 entries, 0 to 98107
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   travel_type      31354 non-null  object 
 1   rating           54837 non-null  float64
 2   label            54837 non-null  object 
 3   combined_review  68292 non-null  object 
 4   date             68292 non-null  object 
 5   covid            68292 non-null  object 
 6   is_local         68292 non-null  int64  
 7   stem_review      68292 non-null  object 
 8   lem_review       68292 non-null  object 
 9   cleaned_review   68292 non-null  object 
 10  star             68292 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 6.3+ MB


,travel_type,rating,label,combined_review,date,covid,is_local,stem_review,lem_review,cleaned_review,star
0,couple,4.0,Positive,clean comfortable hotel expensive find decent ...,2023-08-01,PostCovid,0,clean comfort hotel expens find decent hotel e...,clean comfortable hotel expensive find decent ...,clean comfortable hotel expensive find decent ...,3
1,family,5.0,Positive,good hotel great location great place location...,2023-08-01,PostCovid,0,good hotel great locat great place locat great...,good hotel great location great place location...,good hotel great location great place location...,3
2,friends,5.0,Positive,good place decent price good place good price ...,2022-10-01,PostCovid,0,good place decent price good place good price ...,good place decent price good place good price ...,good place decent price good place good price ...,3
3,solo,5.0,Positive,great location great staff ibis neat tidy hote...,2023-08-01,PostCovid,0,great locat great staff ibi neat tidi hotel li...,great location great staff ibis neat tidy hote...,great location great staff ibis neat tidy hote...,3
4,business,4.0,Positive,good budget stay stayed days nice location sev...,2022-08-01,PostCovid,0,good budget stay stay day nice locat seven ele...,good budget stay stay day nice location seven ...,good budget stay stayed days nice location sev...,3


In [5]:
df3 = data.query('star==3')
df4 = data.query('star==4')
df5 = data.query('star==5')

# Assign LDA topics

In [7]:
lda3 = [(0,
  [('room', 0.028019654),
   ('shower', 0.02429848),
   ('water', 0.02003805),
   ('good', 0.016749734),
   ('bathroom', 0.016249016),
   ('small', 0.015367531),
   ('pool', 0.014033569),
   ('area', 0.01364396),
   ('well', 0.0131019065),
   ('bed', 0.012332016),
   ('hot', 0.012192612),
   ('floor', 0.011816955),
   ('also', 0.01121319),
   ('space', 0.010460705),
   ('free', 0.010154055),
   ('breakfast', 0.00998285),
   ('use', 0.009889977),
   ('one', 0.009070871),
   ('avail', 0.008202699),
   ('high', 0.008031354)]),
 (1,
  [('room', 0.048877396),
   ('hotel', 0.035052437),
   ('small', 0.017294107),
   ('one', 0.015194208),
   ('stay', 0.01469374),
   ('night', 0.014450712),
   ('get', 0.011605806),
   ('us', 0.011314762),
   ('breakfast', 0.010594836),
   ('even', 0.009225123),
   ('book', 0.009213377),
   ('bed', 0.008568587),
   ('could', 0.008473515),
   ('time', 0.0076296735),
   ('day', 0.0072289393),
   ('bad', 0.0071428926),
   ('size', 0.007104914),
   ('like', 0.006724387),
   ('good', 0.0066856826),
   ('would', 0.0066501293)]),
 (2,
  [('hotel', 0.06546004),
   ('good', 0.043226086),
   ('great', 0.033812556),
   ('breakfast', 0.030543184),
   ('stay', 0.028642496),
   ('room', 0.023268161),
   ('walk', 0.022442635),
   ('nice', 0.020002468),
   ('staff', 0.019089388),
   ('clean', 0.018949142),
   ('small', 0.0176249),
   ('pool', 0.01545591),
   ('well', 0.01282588),
   ('comfort', 0.011138362),
   ('food', 0.011052815),
   ('night', 0.010776383),
   ('area', 0.010553828),
   ('excel', 0.010348529),
   ('close', 0.010164016),
   ('also', 0.009887687)]),
 (3,
  [('hotel', 0.061234057),
   ('stay', 0.04743017),
   ('staff', 0.04123435),
   ('room', 0.031478345),
   ('check', 0.030055951),
   ('great', 0.02450151),
   ('us', 0.02277084),
   ('help', 0.021023287),
   ('good', 0.02054956),
   ('time', 0.015681427),
   ('thank', 0.014486463),
   ('nice', 0.012198553),
   ('clean', 0.011541337),
   ('day', 0.010844667),
   ('front', 0.010427587),
   ('would', 0.010189428),
   ('back', 0.0100764185),
   ('come', 0.009442571),
   ('alway', 0.0092536835),
   ('also', 0.00901258)])]

In [8]:
lda4 = [(0,
  [('hotel', 0.06066543),
   ('good', 0.05662687),
   ('great', 0.051247127),
   ('stay', 0.039658293),
   ('pool', 0.036424354),
   ('room', 0.035750892),
   ('nice', 0.03357231),
   ('staff', 0.031510737),
   ('breakfast', 0.030704545),
   ('clean', 0.022773447),
   ('comfort', 0.020109307),
   ('excel', 0.019477336),
   ('well', 0.018372677),
   ('help', 0.013885156),
   ('would', 0.013775486),
   ('food', 0.01340219),
   ('buffet', 0.012523241),
   ('love', 0.010999166),
   ('area', 0.010570749),
   ('night', 0.010390278)]),
 (1,
  [('room', 0.06312547),
   ('check', 0.017642308),
   ('hotel', 0.01647614),
   ('bed', 0.011348137),
   ('bathroom', 0.010589932),
   ('one', 0.010202156),
   ('book', 0.010044039),
   ('shower', 0.009454524),
   ('time', 0.009368337),
   ('good', 0.00930641),
   ('get', 0.00887617),
   ('floor', 0.008515672),
   ('like', 0.00827102),
   ('even', 0.0077712038),
   ('could', 0.0072875964),
   ('night', 0.0071407855),
   ('would', 0.007056457),
   ('us', 0.006762563),
   ('stay', 0.006739076),
   ('work', 0.006467629)]),
 (2,
  [('stay', 0.04730338),
   ('hotel', 0.046276893),
   ('staff', 0.0447593),
   ('us', 0.023029383),
   ('great', 0.021268932),
   ('help', 0.018108215),
   ('time', 0.016622065),
   ('thank', 0.01511681),
   ('excel', 0.0143885),
   ('room', 0.014370245),
   ('alway', 0.01285341),
   ('back', 0.011351997),
   ('day', 0.009717909),
   ('wonder', 0.009487917),
   ('best', 0.009451669),
   ('club', 0.008978278),
   ('love', 0.008878429),
   ('would', 0.008704857),
   ('make', 0.0085479645),
   ('made', 0.008402818)]),
 (3,
  [('airport', 0.06567468),
   ('hotel', 0.06527326),
   ('stay', 0.032393623),
   ('termin', 0.0260096),
   ('flight', 0.025122376),
   ('room', 0.02438863),
   ('night', 0.01972968),
   ('check', 0.019121783),
   ('one', 0.014915458),
   ('get', 0.01162541),
   ('comfort', 0.011361008),
   ('time', 0.010810979),
   ('walk', 0.010133061),
   ('transit', 0.009941516),
   ('long', 0.009932345),
   ('next', 0.0096626645),
   ('morn', 0.009263578),
   ('day', 0.009244583),
   ('overnight', 0.009122408),
   ('late', 0.008109804)]),
 (4,
  [('hotel', 0.06364555),
   ('walk', 0.033848412),
   ('good', 0.026078753),
   ('shop', 0.023754375),
   ('great', 0.022542689),
   ('food', 0.022338388),
   ('also', 0.017887287),
   ('close', 0.015937222),
   ('near', 0.01474164),
   ('station', 0.014280471),
   ('free', 0.013040087),
   ('across', 0.013000975),
   ('area', 0.01297331),
   ('road', 0.012972581),
   ('nice', 0.012285239),
   ('bu', 0.012045242),
   ('town', 0.011958242),
   ('pool', 0.0113485195),
   ('away', 0.010629319),
   ('mall', 0.010490889)])]

In [9]:
lda5 = [(0,
  [('hotel', 0.061920475),
   ('pool', 0.034780033),
   ('view', 0.0328016),
   ('great', 0.028055392),
   ('stay', 0.026859544),
   ('room', 0.026566213),
   ('bay', 0.021528374),
   ('good', 0.01875146),
   ('night', 0.014018549),
   ('nice', 0.013574248),
   ('shop', 0.01347775),
   ('floor', 0.013061163),
   ('breakfast', 0.011596192),
   ('one', 0.009716363),
   ('well', 0.0096779065),
   ('also', 0.009599337),
   ('walk', 0.009585248),
   ('mall', 0.008831545),
   ('food', 0.008742659),
   ('place', 0.008587227)]),
 (1,
  [('food', 0.07006818),
   ('good', 0.050391592),
   ('tea', 0.038146388),
   ('great', 0.037390612),
   ('restaur', 0.029425347),
   ('afternoon', 0.027223198),
   ('dinner', 0.024779718),
   ('buffet', 0.024768097),
   ('swiss', 0.021368127),
   ('nice', 0.019505108),
   ('excel', 0.017836131),
   ('staff', 0.015534015),
   ('attent', 0.011996699),
   ('lunch', 0.011778906),
   ('select', 0.01161212),
   ('love', 0.011317591),
   ('dine', 0.0109803695),
   ('well', 0.010426343),
   ('breakfast', 0.00999563),
   ('come', 0.009959913)]),
 (2,
  [('room', 0.04861536),
   ('check', 0.026507057),
   ('hotel', 0.02609595),
   ('time', 0.014249958),
   ('stay', 0.01398844),
   ('get', 0.0125461705),
   ('one', 0.011784243),
   ('would', 0.010396946),
   ('book', 0.010188739),
   ('night', 0.0097509865),
   ('even', 0.00857634),
   ('pool', 0.008448032),
   ('like', 0.0081832325),
   ('day', 0.008071083),
   ('could', 0.00775153),
   ('floor', 0.007650544),
   ('go', 0.0075336895),
   ('staff', 0.007229006),
   ('bed', 0.0070900335),
   ('wait', 0.0066880765)]),
 (3,
  [('us', 0.059095435),
   ('birthday', 0.02917478),
   ('thank', 0.017529981),
   ('check', 0.01695204),
   ('cake', 0.014356298),
   ('wed', 0.013660884),
   ('also', 0.012656271),
   ('day', 0.012325384),
   ('special', 0.012037271),
   ('team', 0.01169118),
   ('made', 0.011527236),
   ('even', 0.0111270845),
   ('gave', 0.010920424),
   ('park', 0.010272152),
   ('make', 0.010252163),
   ('n', 0.009127772),
   ('wife', 0.008447073),
   ('went', 0.008130026),
   ('help', 0.007957786),
   ('staff', 0.0075411727)]),
 (4,
  [('stay', 0.06459443),
   ('hotel', 0.053955726),
   ('staff', 0.051875144),
   ('great', 0.034243435),
   ('club', 0.024310365),
   ('room', 0.023868028),
   ('help', 0.020169307),
   ('excel', 0.019916743),
   ('time', 0.016955014),
   ('would', 0.014386054),
   ('best', 0.01415547),
   ('back', 0.012771573),
   ('wonder', 0.011836362),
   ('alway', 0.010785513),
   ('us', 0.010444606),
   ('thank', 0.010332707),
   ('well', 0.010328791),
   ('fantast', 0.009910117),
   ('love', 0.009776346),
   ('made', 0.009334151)])]

# Dependency Parsing

In [10]:
import spacy
# Pass in LDA topics output to remove overlap words by choosing highest prob
def restruct_topics(topics): 
    word_prob_dict = {}
    for i in range(len(topics)):
        topic_num = topics[i][0]
        for word, prob in topics[i][1]:
            if word in word_prob_dict:
                word_prob_dict[word].append((topic_num, prob))
            else:
                ls = [(topic_num, prob)]
                word_prob_dict[word] = ls
    new_dict = {}
    for word in word_prob_dict:
        topic, highest_prob = max(word_prob_dict[word], key = lambda x: x[1])
        if topic in new_dict:
            new_dict[topic].append(word)
        else:
            ls = [word]
            new_dict[topic] = ls
    return new_dict

In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
#takes in the dataframe and appends to each row (aspect, review) pair(s)
def dep_parse(data, new_dict):
    data['aspect_sentiment'] = np.empty((len(data), 0)).tolist()
    for i in range(len(data)):
        #print(i)
        sentence = data['combined_review'][i]
        doc = nlp(sentence)
        aspect_sentiment = []
        for word in doc:
            cond = False
            for x in new_dict.values():
                if word.text in x:
                    cond = True
                    break
            if not cond:
                continue
            if word.pos_ == 'NOUN': 
                for j in word.lefts:
                    #print(j, word, j.dep_, j.pos_)
                    if j.dep_ == 'amod' and j.pos_ == 'ADJ':
                        tup = [word, j, list(new_dict.values()).index(x)]
                        #print(tup)
                        aspect_sentiment.append(tup)
                    for k in j.lefts:
                        if k.dep_ == 'advmod':
                            #print(word, j ,k)
                            tup = [word, k.text + ' ' + j.text, list(new_dict.values()).index(x)]
                            aspect_sentiment.append(tup)
                            if (word, j) in aspect_sentiment:
                                aspect_sentiment.remove([word, j, list(new_dict.values()).index(x)])
        data['aspect_sentiment'][i] = aspect_sentiment

In [13]:
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df5 = df5.reset_index(drop=True)

In [15]:
dep_parse(df3, restruct_topics(lda3))

In [16]:
dep_parse(df4, restruct_topics(lda4))

In [17]:
dep_parse(df5, restruct_topics(lda5))

# Textblob

In [21]:
from textblob import TextBlob

## Mean Textblob Function

In [22]:
# Create a function to calculate mean TextBlob polarity for each topic
def calculate_mean_textblob_polarity(row):
    scores = {}
    counts = {}
    
    for aspect_sentiment in row:
        aspect = aspect_sentiment[0]
        adjective = aspect_sentiment[1]
        topic = aspect_sentiment[2]

        # Concatenate aspect and adjective
        text_to_analyze = f"{aspect} {adjective}"

        # Calculate TextBlob polarity
        polarity = TextBlob(text_to_analyze).sentiment.polarity

        # Accumulate polarity values of each aspect within a topic
        column_name = f'topic_{topic}'
        scores[column_name] = scores.get(column_name, 0) + polarity # add polarity to existing polarity (if have)
        # if a topic occurence is more than once, scores[column_name] will be the summed polarity

        # Count occurrences of each aspect within a topic
        count_column_name = f'count_{column_name}'
        counts[count_column_name] = counts.get(count_column_name, 0) + 1
    
    # Calculate mean polarity for each topic
    for count_column_name, topic_count in counts.items():
        #print(counts.items())
        column_name = count_column_name.replace('count_', '')
        mean_polarity = scores[column_name] / counts[count_column_name]
        scores[column_name] = mean_polarity # replace summed polarity with mean polarity

    return pd.Series({**scores})

In [23]:
columns_to_drop = ["combined_review", "date", "stem_review", "lem_review", "star", "cleaned_review"]

In [24]:
result_df3 = df3['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df3["overall_textblob_polarity"] = df3.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df3_with_scores = pd.concat([df3, result_df3], axis=1)

df3_final = df3_with_scores[df3_with_scores['aspect_sentiment'].apply(lambda x: bool(x))] 
print(df3_with_scores.shape)
print(df3_final.shape)
df3_final = df3_final.drop(columns = columns_to_drop)

(14070, 17)
(12331, 17)


In [25]:
df3_final.head()

,travel_type,rating,label,covid,is_local,aspect_sentiment,topic_2,topic_0,topic_1,topic_3,overall_textblob_polarity
0,couple,4.0,Positive,PostCovid,0,"[[hotel, clean, 2], [hotel, comfortable, 2], [...",0.311111,0.143333,0.205556,NaN,0.149749
1,family,5.0,Positive,PostCovid,0,"[[room, great, 0], [room, small, 0], [hotel, g...",0.700000,0.275000,NaN,NaN,0.455000
2,friends,5.0,Positive,PostCovid,0,"[[food, close, 2], [check, easy, 3]]",0.000000,NaN,NaN,0.433333,0.396296
3,solo,5.0,Positive,PostCovid,0,"[[staff, great, 3]]",NaN,NaN,NaN,0.800000,0.312083
4,business,4.0,Positive,PostCovid,0,"[[room, also night, 0], [time, long, 3]]",NaN,0.000000,NaN,-0.050000,0.372917


In [26]:
result_df4 = df4['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df4["overall_textblob_polarity"] = df4.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df4_with_scores = pd.concat([df4, result_df4], axis=1)

df4_final = df4_with_scores[df4_with_scores['aspect_sentiment'].apply(lambda x: bool(x))]
print(df4_with_scores.shape)
print(df4_final.shape)
df4_final = df4_final.drop(columns = columns_to_drop)

(18600, 18)
(16349, 18)


In [27]:
result_df5 = df5['aspect_sentiment'].apply(calculate_mean_textblob_polarity)
result_df5["overall_textblob_polarity"] = df5.apply(lambda row: TextBlob(row["combined_review"]).sentiment.polarity, axis = 1)

# Concatenate the result DataFrame with df_subset
df5_with_scores = pd.concat([df5, result_df5], axis=1)

df5_final = df5_with_scores[df5_with_scores['aspect_sentiment'].apply(lambda x: bool(x))]
print(df5_with_scores.shape)
print(df5_final.shape)
df5_final = df5_final.drop(columns = columns_to_drop)

(35622, 18)
(30972, 18)


In [32]:
csv_name3 = "../../data/textblob/dp_textblob_3_star.csv"
df3_final.to_csv(csv_name3, index=False)

csv_name4 = "../../data/textblob/dp_textblob_4_star.csv"
df4_final.to_csv(csv_name4, index=False)

csv_name5 = "../../data/textblob/dp_textblob_5_star.csv"
df5_final.to_csv(csv_name5, index=False)